In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Notebook 2: Demonstração dos Métodos de Detecção\n",
    "\n",
    "Este notebook demonstra o uso dos dois métodos implementados para a detecção de landmarks:\n",
    "1. **Método Geométrico:** Baseado em características como extremos e curvatura.\n",
    "2. **Método de Machine Learning (ML):** Utiliza um classificador Random Forest treinado.\n",
    "\n",
    "**Objetivos:**\n",
    "1. Carregar e pré-processar uma malha (usando `MeshProcessor`).\n",
    "2. Instanciar e executar o `GeometricDetector`.\n",
    "3. Visualizar os landmarks detectados pelo método geométrico.\n",
    "4. Instanciar e executar o `MLDetector` (se modelos estiverem disponíveis).\n",
    "5. Visualizar os landmarks detectados pelo método ML.\n",
    "6. Comparar visualmente os resultados dos dois métodos."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Configurações Iniciais e Imports\n",
    "import os\n",
    "import sys\n",
    "import logging\n",
    "import time\n",
    "import trimesh\n",
    "import numpy as np\n",
    "import joblib\n",
    "\n",
    "# Adicionar o diretório raiz do projeto ao path\n",
    "module_path = os.path.abspath(os.path.join(\"..\"))\n",
    "if module_path not in sys.path:\n",
    "    sys.path.append(module_path)\n",
    "\n",
    "from src.core.mesh_processor import MeshProcessor\n",
    "from src.core.detector_geometric import GeometricDetector\n",
    "from src.core.detector_ml import MLDetector\n",
    "from src.core.landmarks import LANDMARK_NAMES\n",
    "from src.utils.visualization import plot_landmarks, plot_landmarks_2d\n",
    "from src.utils.helpers import setup_logging, save_landmarks_to_json\n",
    "\n",
    "# Configurar logging\n",
    "setup_logging(log_level=logging.INFO)\n",
    "\n",
    "print(f\"Diretório de trabalho: {os.getcwd()}\")\n",
    "print(f\"Path do módulo: {module_path}\")\n",
    "print(f\"Landmarks definidos: {len(LANDMARK_NAMES)}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Configurar diretórios\n",
    "BASE_DIR = module_path\n",
    "DATA_DIR = os.path.join(BASE_DIR, \"data\", \"skulls\")\n",
    "CACHE_DIR = os.path.join(BASE_DIR, \"data\", \"cache\")\n",
    "RESULTS_DIR = os.path.join(BASE_DIR, \"results\")\n",
    "MODEL_DIR = os.path.join(BASE_DIR, \"models\")\n",
    "\n",
    "# Criar diretórios se não existirem\n",
    "for directory in [DATA_DIR, CACHE_DIR, RESULTS_DIR, MODEL_DIR]:\n",
    "    os.makedirs(directory, exist_ok=True)\n",
    "\n",
    "print(f\"Diretórios configurados:\")\n",
    "print(f\"  Dados: {DATA_DIR}\")\n",
    "print(f\"  Cache: {CACHE_DIR}\")\n",
    "print(f\"  Resultados: {RESULTS_DIR}\")\n",
    "print(f\"  Modelos ML: {MODEL_DIR}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Criar arquivo STL dummy para demonstração\n",
    "dummy_stl_filename = \"dummy_skull_demo.stl\"\n",
    "dummy_stl_path = os.path.join(DATA_DIR, dummy_stl_filename)\n",
    "\n",
    "if not os.path.exists(dummy_stl_path):\n",
    "    print(f\"Criando arquivo STL dummy em {dummy_stl_path}\")\n",
    "    # Criar esfera com deformações para simular crânio\n",
    "    mesh_dummy = trimesh.primitives.Sphere(radius=70, subdivisions=4)\n",
    "    mesh_dummy.apply_scale([0.8, 1, 0.9])  # Deformar um pouco\n",
    "    mesh_dummy.vertices += [10, 0, 70]  # Deslocar\n",
    "    mesh_dummy.export(dummy_stl_path)\n",
    "    print(f\"✅ Arquivo STL dummy '{dummy_stl_filename}' criado\")\n",
    "else:\n",
    "    print(f\"✅ Usando arquivo STL existente: '{dummy_stl_filename}'\")\n",
    "\n",
    "TARGET_STL_FILENAME = dummy_stl_filename\n",
    "print(f\"Arquivo alvo: {TARGET_STL_FILENAME}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Carregar e Pré-processar a Malha\n",
    "\n",
    "Carregamos a malha de demonstração e aplicamos a simplificação."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Inicializar processador\n",
    "processor = MeshProcessor(data_dir=DATA_DIR, cache_dir=CACHE_DIR)\n",
    "TARGET_FACES = 2000  # Simplificar para melhor desempenho\n",
    "\n",
    "print(f\"Carregando e processando: {TARGET_STL_FILENAME}\")\n",
    "\n",
    "# Carregar malha original\n",
    "start_time = time.time()\n",
    "mesh_original = processor.load_skull(TARGET_STL_FILENAME, use_cache=True)\n",
    "load_time = time.time() - start_time\n",
    "\n",
    "if not mesh_original:\n",
    "    print(f\"❌ Falha ao carregar {TARGET_STL_FILENAME}\")\n",
    "    raise Exception(\"Não é possível continuar sem malha\")\n",
    "\n",
    "print(f\"✅ Malha carregada em {load_time:.4f}s\")\n",
    "print(f\"   Vértices originais: {len(mesh_original.vertices):,}\")\n",
    "print(f\"   Faces originais: {len(mesh_original.faces):,}\")\n",
    "\n",
    "# Simplificar malha\n",
    "start_time = time.time()\n",
    "mesh_simplified = processor.simplify(\n",
    "    mesh_original, \n",
    "    target_faces=TARGET_FACES, \n",
    "    use_cache=True, \n",
    "    original_filename=TARGET_STL_FILENAME\n",
    ")\n",
    "simplify_time = time.time() - start_time\n",
    "\n",
    "if not mesh_simplified:\n",
    "    print(\"⚠️  Falha na simplificação, usando malha original\")\n",
    "    mesh_simplified = mesh_original\n",
    "else:\n",
    "    print(f\"✅ Malha simplificada em {simplify_time:.4f}s\")\n",
    "    print(f\"   Vértices simplificados: {len(mesh_simplified.vertices):,}\")\n",
    "    print(f\"   Faces simplificadas: {len(mesh_simplified.faces):,}\")\n",
    "    reduction = (1 - len(mesh_simplified.faces) / len(mesh_original.faces)) * 100\n",
    "    print(f\"   Redução: {reduction:.1f}%\")\n",
    "\n",
    "print(f\"\\n📊 Malha para detecção pronta: {len(mesh_simplified.vertices):,} vértices, {len(mesh_simplified.faces):,} faces\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Detecção com Método Geométrico\n",
    "\n",
    "Instanciamos `GeometricDetector` e aplicamos à malha simplificada."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"=== Executando Detecção Geométrica ===\")\n",
    "\n",
    "# Inicializar detector geométrico\n",
    "detector_geom = GeometricDetector()\n",
    "\n",
    "# Executar detecção\n",
    "start_time = time.time()\n",
    "landmarks_geometric = detector_geom.detect(mesh_simplified)\n",
    "detection_time = time.time() - start_time\n",
    "\n",
    "if landmarks_geometric:\n",
    "    # Contar landmarks detectados\n",
    "    detected_count = sum(1 for coords in landmarks_geometric.values() if coords is not None)\n",
    "    total_count = len(landmarks_geometric)\n",
    "    \n",
    "    print(f\"✅ Detecção geométrica concluída em {detection_time:.4f}s\")\n",
    "    print(f\"📊 Landmarks detectados: {detected_count}/{total_count} ({detected_count/total_count*100:.1f}%)\")\n",
    "    \n",
    "    print(\"\\n--- Landmarks Detectados (Geométrico) ---\")\n",
    "    for name, coords in landmarks_geometric.items():\n",
    "        if coords is not None:\n",
    "            coord_str = f\"[{coords[0]:.2f}, {coords[1]:.2f}, {coords[2]:.2f}]\"\n",
    "            print(f\"  ✅ {name}: {coord_str}\")\n",
    "        else:\n",
    "            print(f\"  ❌ {name}: Não detectado\")\n",
    "    \n",
    "    # Salvar resultados\n",
    "    geom_result_path = os.path.join(\n",
    "        RESULTS_DIR, \n",
    "        f\"{os.path.splitext(TARGET_STL_FILENAME)[0]}_geometric_landmarks.json\"\n",
    "    )\n",
    "    if save_landmarks_to_json(landmarks_geometric, geom_result_path):\n",
    "        print(f\"💾 Resultados geométricos salvos em: {geom_result_path}\")\n",
    "    \n",
    "else:\n",
    "    print(\"❌ Falha na detecção geométrica\")\n",
    "    landmarks_geometric = None"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Detecção com Método de Machine Learning\n",
    "\n",
    "Instanciamos `MLDetector` e tentamos carregar modelos pré-treinados. Se não estiverem disponíveis, criamos modelos dummy básicos para demonstração."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"=== Verificando Disponibilidade de Modelos ML ===\")\n",
    "\n",
    "# Inicializar detector ML\n",
    "ml_detector = MLDetector(model_dir=MODEL_DIR)\n",
    "\n",
    "# Verificar quais modelos estão disponíveis\n",
    "available_models = []\n",
    "for landmark_name in LANDMARK_NAMES:\n",
    "    model_path = os.path.join(MODEL_DIR, f\"rf_model_{landmark_name}.joblib\")\n",
    "    scaler_path = os.path.join(MODEL_DIR, f\"scaler_{landmark_name}.joblib\")\n",
    "    if os.path.exists(model_path) and os.path.exists(scaler_path):\n",
    "        available_models.append(landmark_name)\n",
    "\n",
    "print(f\"📊 Modelos ML disponíveis: {len(available_models)}/{len(LANDMARK_NAMES)}\")\n",
    "\n",
    "if available_models:\n",
    "    print(\"✅ Modelos encontrados para:\")\n",
    "    for model in available_models:\n",
    "        print(f\"   - {model}\")\n",
    "else:\n",
    "    print(\"⚠️  Nenhum modelo ML encontrado\")\n",
    "    print(\"🔧 Criando modelos dummy básicos para demonstração...\")\n",
    "    \n",
    "    # Criar dados de treinamento dummy muito simples\n",
    "    dummy_mesh_train = trimesh.primitives.Sphere(radius=50, center=[0, 0, 50])\n",
    "    dummy_gt_train = {\n",
    "        \"Glabela\": [0, 50, 50], \n",
    "        \"Nasion\": [0, 45, 40], \n",
    "        \"Bregma\": [0, 0, 100], \n",
    "        \"Opisthocranion\": [0, -50, 50], \n",
    "        \"Euryon_Esquerdo\": [-50, 0, 50], \n",
    "        \"Euryon_Direito\": [50, 0, 50], \n",
    "        \"Vertex\": [0, 0, 100], \n",
    "        \"Inion\": [0, -45, 40]\n",
    "    }\n",
    "    \n",
    "    # Treinar modelos dummy apenas para alguns landmarks principais\n",
    "    primary_landmarks = [\"Glabela\", \"Bregma\", \"Euryon_Direito\"]\n",
    "    training_successful = True\n",
    "    \n",
    "    for name in primary_landmarks:\n",
    "        if name in dummy_gt_train:\n",
    "            print(f\"   🔧 Treinando modelo dummy para: {name}\")\n",
    "            try:\n",
    "                success = ml_detector.train([dummy_mesh_train], [dummy_gt_train], name)\n",
    "                if success:\n",
    "                    available_models.append(name)\n",
    "                    print(f\"      ✅ Modelo {name} criado\")\n",
    "                else:\n",
    "                    print(f\"      ❌ Falha ao criar modelo {name}\")\n",
    "                    training_successful = False\n",
    "            except Exception as e:\n",
    "                print(f\"      ❌ Erro ao treinar {name}: {e}\")\n",
    "                training_successful = False\n",
    "    \n",
    "    if training_successful and available_models:\n",
    "        print(f\"✅ Modelos dummy criados com sucesso para {len(available_models)} landmarks\")\n",
    "    else:\n",
    "        print(\"❌ Falha ao criar modelos dummy\")\n",
    "\n",
    "print(f\"\\n📊 Total de modelos disponíveis: {len(available_models)}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"=== Executando Detecção por Machine Learning ===\")\n",
    "\n",
    "landmarks_ml = None\n",
    "\n",
    "if available_models:\n",
    "    # Executar detecção ML\n",
    "    start_time = time.time()\n",
    "    landmarks_ml = ml_detector.detect(mesh_simplified)\n",
    "    ml_detection_time = time.time() - start_time\n",
    "    \n",
    "    if landmarks_ml:\n",
    "        # Contar landmarks detectados\n",
    "        ml_detected_count = sum(1 for coords in landmarks_ml.values() if coords is not None)\n",
    "        \n",
    "        print(f\"✅ Detecção ML concluída em {ml_detection_time:.4f}s\")\n",
    "        print(f\"📊 Landmarks detectados: {ml_detected_count}/{len(landmarks_ml)} ({ml_detected_count/len(landmarks_ml)*100:.1f}%)\")\n",
    "        \n",
    "        print(\"\\n--- Landmarks Detectados (Machine Learning) ---\")\n",
    "        for name, coords in landmarks_ml.items():\n",
    "            if coords is not None:\n",
    "                coord_str = f\"[{coords[0]:.2f}, {coords[1]:.2f}, {coords[2]:.2f}]\"\n",
    "                print(f\"  ✅ {name}: {coord_str}\")\n",
    "            else:\n",
    "                print(f\"  ❌ {name}: Não detectado / Modelo ausente\")\n",
    "        \n",
    "        # Salvar resultados\n",
    "        ml_result_path = os.path.join(\n",
    "            RESULTS_DIR, \n",
    "            f\"{os.path.splitext(TARGET_STL_FILENAME)[0]}_ml_landmarks.json\"\n",
    "        )\n",
    "        if save_landmarks_to_json(landmarks_ml, ml_result_path):\n",
    "            print(f\"💾 Resultados ML salvos em: {ml_result_path}\")\n",
    "        \n",
    "    else:\n",
    "        print(\"❌ Falha na detecção ML\")\n",
    "        \n",
    "else:\n",
    "    print(\"❌ Não é possível executar detecção ML sem modelos treinados\")\n",
    "    print(\"💡 Para usar ML, treine modelos primeiro com dados reais\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Visualização Comparativa\n",
    "\n",
    "Visualizamos os resultados de ambos os métodos lado a lado usando projeções 2D."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"=== Gerando Visualizações Comparativas ===\")\n",
    "\n",
    "# Visualização do método geométrico\n",
    "if landmarks_geometric:\n",
    "    print(\"🎨 Gerando visualização geométrica...\")\n",
    "    vis_geom_path = os.path.join(\n",
    "        RESULTS_DIR, \n",
    "        f\"{os.path.splitext(TARGET_STL_FILENAME)[0]}_geometric_visualization_2d.png\"\n",
    "    )\n",
    "    \n",
    "    success_geom = plot_landmarks_2d(\n",
    "        mesh_simplified, \n",
    "        landmarks_geometric, \n",
    "        title=f\"Método Geométrico - {TARGET_STL_FILENAME}\", \n",
    "        save_path=vis_geom_path\n",
    "    )\n",
    "    \n",
    "    if success_geom:\n",
    "        print(f\"✅ Visualização geométrica salva em: {vis_geom_path}\")\n",
    "        \n",
    "        # Exibir no notebook\n",
    "        try:\n",
    "            from IPython.display import Image, display\n",
    "            print(\"\\n=== Resultado Método Geométrico ===\")\n",
    "            display(Image(filename=vis_geom_path))\n",
    "        except:\n",
    "            print(\"Não foi possível exibir a imagem no notebook\")\n",
    "    else:\n",
    "        print(\"❌ Falha na visualização geométrica\")\n",
    "\n",
    "# Visualização do método ML\n",
    "if landmarks_ml:\n",
    "    print(\"\\n🎨 Gerando visualização ML...\")\n",
    "    vis_ml_path = os.path.join(\n",
    "        RESULTS_DIR, \n",
    "        f\"{os.path.splitext(TARGET_STL_FILENAME)[0]}_ml_visualization_2d.png\"\n",
    "    )\n",
    "    \n",
    "    success_ml = plot_landmarks_2d(\n",
    "        mesh_simplified, \n",
    "        landmarks_ml, \n",
    "        title=f\"Método Machine Learning - {TARGET_STL_FILENAME}\", \n",
    "        save_path=vis_ml_path\n",
    "    )\n",
    "    \n",
    "    if success_ml:\n",
    "        print(f\"✅ Visualização ML salva em: {vis_ml_path}\")\n",
    "        \n",
    "        # Exibir no notebook\n",
    "        try:\n",
    "            from IPython.display import Image, display\n",
    "            print(\"\\n=== Resultado Método Machine Learning ===\")\n",
    "            display(Image(filename=vis_ml_path))\n",
    "        except:\n",
    "            print(\"Não foi possível exibir a imagem no notebook\")\n",
    "    else:\n",
    "        print(\"❌ Falha na visualização ML\")\n",
    "\n",
    "# Caso nenhum método tenha funcionado\n",
    "if not landmarks_geometric and not landmarks_ml:\n",
    "    print(\"❌ Nenhum método de detecção produziu resultados válidos\")\n",
    "elif not landmarks_geometric:\n",
    "    print(\"⚠️  Apenas o método ML produziu resultados\")\n",
    "elif not landmarks_ml:\n",
    "    print(\"⚠️  Apenas o método geométrico produziu resultados\")\n",
    "else:\n",
    "    print(\"✅ Ambos os métodos produziram resultados para comparação\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Análise Comparativa dos Resultados\n",
    "\n",
    "Comparamos quantitativamente os resultados dos dois métodos."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"=== Análise Comparativa dos Métodos ===\")\n",
    "\n",
    "if landmarks_geometric and landmarks_ml:\n",
    "    print(\"\\n📊 Comparação de Performance:\")\n",
    "    \n",
    "    # Taxa de detecção\n",
    "    geom_detected = sum(1 for coords in landmarks_geometric.values() if coords is not None)\n",
    "    ml_detected = sum(1 for coords in landmarks_ml.values() if coords is not None)\n",
    "    total_landmarks = len(LANDMARK_NAMES)\n",
    "    \n",
    "    print(f\"Taxa de Detecção:\")\n",
    "    print(f\"  Geométrico: {geom_detected}/{total_landmarks} ({geom_detected/total_landmarks*100:.1f}%)\")\n",
    "    print(f\"  ML: {ml_detected}/{total_landmarks} ({ml_detected/total_landmarks*100:.1f}%)\")\n",
    "    \n",
    "    # Landmarks detectados por ambos\n",
    "    both_detected = []\n",
    "    only_geom = []\n",
    "    only_ml = []\n",
    "    \n",
    "    for name in LANDMARK_NAMES:\n",
    "        geom_found = landmarks_geometric.get(name) is not None\n",
    "        ml_found = landmarks_ml.get(name) is not None\n",
    "        \n",
    "        if geom_found and ml_found:\n",
    "            both_detected.append(name)\n",
    "        elif geom_found:\n",
    "            only_geom.append(name)\n",
    "        elif ml_found:\n",
    "            only_ml.append(name)\n",
    "    \n",
    "    print(f\"\\n🎯 Concordância entre Métodos:\")\n",
    "    print(f\"  Detectados por ambos: {len(both_detected)} ({both_detected})\")\n",
    "    print(f\"  Apenas geométrico: {len(only_geom)} ({only_geom})\")\n",
    "    print(f\"  Apenas ML: {len(only_ml)} ({only_ml})\")\n",
    "    \n",
    "    # Distâncias entre detecções correspondentes\n",
    "    if both_detected:\n",
    "        print(f\"\\n📏 Distâncias entre Detecções Correspondentes:\")\n",
    "        for name in both_detected:\n",
    "            geom_coords = np.array(landmarks_geometric[name])\n",
    "            ml_coords = np.array(landmarks_ml[name])\n",
    "            distance = np.linalg.norm(geom_coords - ml_coords)\n",
    "            print(f\"  {name}: {distance:.2f} mm\")\n",
    "    \n",
    "    # Tempo de processamento\n",
    "    if 'detection_time' in locals() and 'ml_detection_time' in locals():\n",
    "        print(f\"\\n⏱️  Tempo de Processamento:\")\n",
    "        print(f\"  Geométrico: {detection_time:.4f}s\")\n",
    "        print(f\"  ML: {ml_detection_time:.4f}s\")\n",
    "        if detection_time > 0:\n",
    "            speedup = ml_detection_time / detection_time\n",
    "            if speedup > 1:\n",
    "                print(f\"  Geométrico é {speedup:.1f}x mais rápido\")\n",
    "            else:\n",
    "                print(f\"  ML é {1/speedup:.1f}x mais rápido\")\n",
    "\n",
    "elif landmarks_geometric:\n",
    "    print(\"✅ Apenas método geométrico produziu resultados\")\n",
    "    print(f\"   Taxa de detecção: {geom_detected}/{len(LANDMARK_NAMES)} ({geom_detected/len(LANDMARK_NAMES)*100:.1f}%)\")\n",
    "    \n",
    "elif landmarks_ml:\n",
    "    print(\"✅ Apenas método ML produziu resultados\")\n",
    "    print(f\"   Taxa de detecção: {ml_detected}/{len(LANDMARK_NAMES)} ({ml_detected/len(LANDMARK_NAMES)*100:.1f}%)\")\n",
    "    \n",
    "else:\n",
    "    print(\"❌ Nenhum método produziu resultados válidos\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Conclusão da Demonstração\n",
    "\n",
    "Este notebook demonstrou como utilizar os dois métodos de detecção implementados no sistema.\n",
    "\n",
    "### Principais Observações:\n",
    "\n",
    "#### Método Geométrico:\n",
    "- ✅ **Vantagens:** Rápido, não requer treinamento, baseado em princípios anatômicos\n",
    "- ⚠️ **Limitações:** Pode ser menos preciso, dependente de heurísticas\n",
    "\n",
    "#### Método Machine Learning:\n",
    "- ✅ **Vantagens:** Potencialmente mais preciso com dados adequados, adaptável\n",
    "- ⚠️ **Limitações:** Requer modelos treinados, dependente da qualidade dos dados\n",
    "\n",
    "### Recomendações:\n",
    "\n",
    "1. **Para uso imediato:** Use o método geométrico que não requer treinamento\n",
    "2. **Para melhor precisão:** Treine modelos ML com dados reais representativos\n",
    "3. **Para robustez:** Combine ambos os métodos e use votação ou consenso\n",
    "\n",
    "### Próximos Passos:\n",
    "\n",
    "- O próximo notebook (`03_analise_resultados.ipynb`) focará na avaliação quantitativa\n",
    "- Para usar ML efetivamente, é necessário treinar com dados ground truth reais\n",
    "- As visualizações ajudam a validar a qualidade das detecções\n",
    "\n",
    "**Ambos os métodos foram demonstrados com sucesso e estão prontos para uso.**"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.0"
  }\n },\n \"nbformat\": 4,\n \"nbformat_minor\": 4\n}"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}